In [1]:
import os 
PIPELINE_NAME = "iris-simple"
PIPELINE_ROOT = os.path.join("pipelines", PIPELINE_NAME)
METADATA_PATH = os.path.join("metadata", PIPELINE_NAME, "metadata.db")
SERVING_MODEL_DIR = os.path.join("serving_model", PIPELINE_NAME)
DATA_ROOT = os.path.join("iris_data")

In [2]:
from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [3]:
# TODO Long, move to Transformer
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(data[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]])
# le = LabelEncoder()
# processed_data = pd.DataFrame(data=scaled_data, columns=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"])
# processed_data["Species"] = le.fit_transform(data["Species"].to_numpy().reshape(-1, 1))
# processed_data.to_csv("./iris_data/processed_iris_data.csv")

In [4]:
import urllib.request
import tempfile

_data_filepath = os.path.join("iris_data", "processed_iris_data.csv")

In [5]:
import pandas as pd
data = pd.read_csv(_data_filepath)
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,-0.900681,1.032057,-1.341272,-1.312977,0
1,-1.143017,-0.124958,-1.341272,-1.312977,0
2,-1.385353,0.337848,-1.398138,-1.312977,0
3,-1.506521,0.106445,-1.284407,-1.312977,0
4,-1.021849,1.263460,-1.341272,-1.312977,0


In [6]:
import tfx 

def create_pipeline(
    pipeline_name, 
    pipeline_root, 
    data_root, 
    module_file, 
    serving_model_dir, 
    metadata_path): 
    # Make sure directory has no other hidden files
    example_gen = tfx.components.CsvExampleGen(input_base=data_root)
    
    trainer = tfx.components.Trainer(
        module_file=module_file,
        examples= example_gen.outputs["examples"],
        train_args=tfx.proto.TrainArgs(splits=["train"], num_steps=40),
        eval_args=tfx.proto.EvalArgs(splits=["train"], num_steps=5)
    )
    
    pusher = tfx.components.Pusher(
        model=trainer.outputs["model"],
        push_destination=tfx.proto.PushDestination(
            filesystem=tfx.proto.PushDestination.Filesystem(base_directory=serving_model_dir)
        )
    )
    
    components = [
        example_gen,
        trainer,
        pusher
    ]
    
    pipeline = tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components
    )
    
    return pipeline

In [7]:
from tfx import v1 as tfx
from tfx_bsl.public import tfxio

2022-06-16 09:48:06.102556: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-16 09:48:06.102579: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is not available: No module named 'tensorflow_decision_forests'
INFO:absl:struct2tensor is not available: No module named 'struct2tensor'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'


In [8]:
tfx.orchestration.LocalDagRunner().run(
    create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file="iris_trainer.py", 
        serving_model_dir=SERVING_MODEL_DIR,
        metadata_path=METADATA_PATH
    )
)

INFO:absl:Generating ephemeral wheel package for '/home/long/Desktop/dev/learning/google/tfx/basics/iris_pj/iris_trainer.py' (including modules: ['pipeline_helper', 'iris_trainer', 'iris_util']).
INFO:absl:User module package has hash fingerprint version f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404.
INFO:absl:Executing: ['/home/long/anaconda3/envs/tfx/bin/python', '/tmp/tmpwjfq65wb/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpi5y8o5n_', '--dist-dir', '/tmp/tmp066q7sg4']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying pipeline_helper.py -> build/lib
copying iris_trainer.py -> build/lib
copying iris_util.py -> build/lib
installing to /tmp/tmpi5y8o5n_
running install
running install_lib
copying build/lib/pipeline_helper.py -> /tmp/tmpi5y8o5n_
copying build/lib/iris_trainer.py -> /tmp/tmpi5y8o5n_
copying build/lib/iris_util.py -> /tmp/tmpi5y8o5n_
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpi5y8o5n_/tfx_user_code_Trainer-0.0+f2564ed

INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris-simple/_wheels/tfx_user_code_Trainer-0.0+f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404-py3-none-any.whl'; target user module is 'iris_trainer'.
INFO:absl:Full user module path is 'iris_trainer@pipelines/iris-simple/_wheels/tfx_user_code_Trainer-0.0+f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "Pusher"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.pusher.executor.Executor"
    }
  }
}
executor_specs {
  key: "Trainer"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.trainer.executor.GenericExecutor"
    }
  }
}
custom_driver_spec

INFO:absl:Processing input csv data iris_data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 71 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/iris-simple/CsvExampleGen/examples/71"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:12071,xor_checksum:1651638713,sum_checksum:1651638713"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "iris-simple:2022-06-16T09:52:31.291877:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties 

INFO:absl:udf_utils.get_fn {'module_path': 'iris_trainer@pipelines/iris-simple/_wheels/tfx_user_code_Trainer-0.0+f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404-py3-none-any.whl', 'eval_args': '{\n  "num_steps": 5,\n  "splits": [\n    "train"\n  ]\n}', 'train_args': '{\n  "num_steps": 40,\n  "splits": [\n    "train"\n  ]\n}', 'custom_config': 'null'} 'run_fn'
INFO:absl:Installing 'pipelines/iris-simple/_wheels/tfx_user_code_Trainer-0.0+f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/long/anaconda3/envs/tfx/bin/python', '-m', 'pip', 'install', '--target', '/tmp/tmpvlerbilu', 'pipelines/iris-simple/_wheels/tfx_user_code_Trainer-0.0+f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404-py3-none-any.whl']


...

INFO:absl:Successfully installed 'pipelines/iris-simple/_wheels/tfx_user_code_Trainer-0.0+f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:Feature PetalLengthCm has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature PetalWidthCm has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature SepalLengthCm has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature SepalWidthCm has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Species has a shape dim {
  size: 1
}
. Setting to DenseTensor.
2022-06-16 09:53:14.024812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 09:53:14.025324: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dler

40/40 [==============================] - 6s 4ms/step - loss: 1.5559 - sparse_categorical_accuracy: 0.1589 - val_loss: 0.9759 - val_sparse_categorical_accuracy: 0.4900


2022-06-16 09:53:27.506242: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: pipelines/iris-simple/Trainer/model/72/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/iris-simple/Trainer/model/72/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/iris-simple/Trainer/model/72/Format-Serving. ModelRun written to pipelines/iris-simple/Trainer/model_run/72
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 72 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model': [Artifact(artifact: uri: "pipelines/iris-simple/Trainer/model/72"
custom_properties {
  key: "name"
  value {
    string_value: "iris-simple:2022-06-16T09:52:31.291877:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.6.1"
  }
}
, artifact_type: name: "Model"
)], 'model_run': [Artifact(artifact: uri: "pipelines/iris-simple/Trainer/model_run/72"
custom_properties {
  key: "name"
  value {
    string_value: "iris-simple:2022-06-16T09:52:31.291877:Trainer:model_run:0"
  }
}
custom_propert